In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark


Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:13 https://developer.

In [2]:
# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()


In [3]:
#Download the Postgres driver to connect tothe 
!wget https://jdbc.postgresql.org/download/postgresql-42.2.20.jar

--2021-05-11 13:05:46--  https://jdbc.postgresql.org/download/postgresql-42.2.20.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1005347 (982K) [application/java-archive]
Saving to: ‘postgresql-42.2.20.jar’

postgresql-42.2.20. 100%[===================>] 981.78K  1.69MB/s    in 0.6s    

2021-05-11 13:05:47 (1.69 MB/s) - ‘postgresql-42.2.20.jar’ saved [1005347/1005347]



In [4]:
 from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.20.jar").getOrCreate()

**Postgres Setup : Read from AWS RDS instance**

In [10]:
# Read from db
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://amznreviews.cx02tixyipyg.us-east-2.rds.amazonaws.com:5432/amznreviews_db"
config = {"user":"root", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

In [11]:
 pet_review = spark.read.jdbc(url=jdbc_url,table="vine_table", properties=config)


**Filter by helpful votes**

In [12]:
# Filter by popular votes
df2 = pet_review.filter(pet_review["total_votes"] >= 20)
df4 = df2.filter(df2["helpful_votes"]/df2["total_votes"] >= 0.5)

In [27]:
df4.describe().show(10)


+-------+--------------+------------------+------------------+------------------+-----+
|summary|     review_id|       star_rating|     helpful_votes|       total_votes| vine|
+-------+--------------+------------------+------------------+------------------+-----+
|  count|         38010|             38010|             38010|             38010|38010|
|   mean|          null|3.7654827676927125| 57.52799263351749| 61.89118652986056| null|
| stddev|          null|1.6037158024563942|107.60446016939757|111.92172613652941| null|
|    min|R10037MJ9LKCVM|                 1|                10|                20|    N|
|    max| RZZMYQ9G22DY0|                 5|              5735|              5811|    Y|
+-------+--------------+------------------+------------------+------------------+-----+



**Describe stats**

In [13]:
from pyspark.sql.functions import col, avg
paid_df = df4.filter(df4['vine']== 'Y')
unpaid_df = df4.filter(df4['vine']== 'N')

paid_df.describe().show()

+-------+--------------+-----------------+------------------+------------------+----+
|summary|     review_id|      star_rating|     helpful_votes|       total_votes|vine|
+-------+--------------+-----------------+------------------+------------------+----+
|  count|           170|              170|               170|               170| 170|
|   mean|          null|3.929411764705882|63.641176470588235| 68.03529411764706|null|
| stddev|          null| 1.11247083912708|107.53304464635706|110.81862649267782|null|
|    min|R10A5XNAP2Y4N7|                1|                13|                20|   Y|
|    max|  RZLBU14F5Y7N|                5|              1239|              1277|   Y|
+-------+--------------+-----------------+------------------+------------------+----+



In [14]:
unpaid_df.describe().show()

+-------+--------------+------------------+------------------+------------------+-----+
|summary|     review_id|       star_rating|     helpful_votes|       total_votes| vine|
+-------+--------------+------------------+------------------+------------------+-----+
|  count|         37840|             37840|             37840|             37840|37840|
|   mean|          null|3.7647463002114163| 57.50052854122622|61.863583509513745| null|
| stddev|          null|1.6055561346315705|107.60541722886963| 111.9273463229035| null|
|    min|R10037MJ9LKCVM|                 1|                10|                20|    N|
|    max| RZZMYQ9G22DY0|                 5|              5735|              5811|    N|
+-------+--------------+------------------+------------------+------------------+-----+



**Paid and Unpaid Percents**

In [15]:

paid_five_star_number = paid_df[paid_df['star_rating']== 5].count()
paid_number = paid_df.count()
percentage_five_star_vine = float(paid_five_star_number) / float(paid_number)
print("Paid Number :", paid_number)
print("paid_five_star_number :" , paid_five_star_number)
print("Percent 5 star mumber (paid)", percentage_five_star_vine)

Paid Number : 170
paid_five_star_number : 65
Percent 5 star mumber (paid) 0.38235294117647056


In [16]:
unpaid_five_star_number = unpaid_df[unpaid_df['star_rating']== 5].count()
unpaid_number = unpaid_df.count()
percentage_five_star_vine = float(unpaid_five_star_number) / float(unpaid_number)
print("Unpaid Number :", unpaid_number)
print("Unpaid_five_star_number :" , unpaid_five_star_number)
print("Percent 5 star mumber (unpaid)", percentage_five_star_vine)

Unpaid Number : 37840
Unpaid_five_star_number : 20612
Percent 5 star mumber (unpaid) 0.5447145877378435
